# Berman Code Star Product

## DecimaltoFn: 
Converting a decimal number into a n-th basis number

In [1]:
#Converting a decimal number into a n-th basis number
#param: number to be converted, basis
import numpy as np

def decimalToFn(num,n):
    ans = np.array([num%n],int)
    if(num > n-1):

        ans = np.concatenate((decimalToFn(num//n,n),ans))
    return ans
#Testing the code output
print(decimalToFn(126,5))

[1 0 0 1]


## FntoDecimal:
Converting the n-th basis vector to a decimal number

In [2]:
#Converting the n-th basis vector to a decimal number
#param: vector containing the decimal number in the n-th basis form, basis value
def FnToDecimal(vec,n):
    ans = 0
    for i in range(0,len(vec)):
        ans+= (vec[len(vec)-i-1]*(n**i))
    return ans
#vec = [2,2,2]
#2.3^0 + 2*3^1 + 2*3^2 = 26
#print(FnToDecimal(vec,3))

## Hweight: 
Finding the hamming weight of a vector

Finding the number of non zero positions in a vector

In [3]:
#Finding the hamming weight of a vector
#Finding the number of non zero positions in a vector
#param: Vector containing some zero positions and some non zero positions
def Hweight(vec):
    #ans = no of non zero positions in a vector
    ans = 0
    n = len(vec)
    
    #supportVec vector: Returns a list of positions from vec which are non zero. Initialized to -1 (assuming the vec has no non zero values in it)
    supp = np.array([-1])
    suppInv = np.array([-1])
    #k1, k2 are flag variables which tells us if the corresponding return array is empty
    k1 = 0 #flag for supp vector
    k2 = 0 #flag for suppInv vector
    for i in range(0,n):
        if(vec[i]!=0):
            ans+=1
            #If nothing has been added to the array, change flag big and add position into res list
            if(k1==0):
                k1=1
                supp = np.array([i])
            else :
                supp = np.concatenate([supp,np.array([i])])
        else:
            if(k2==0):
                k2=1
                suppInv = np.array([i])
            else:
                suppInv = np.concatenate([suppInv,np.array([i])])
    return ans,supp, suppInv
# print(Hweight([1,1,1,1,2,2]))

## vectorSubset:

Given a vector j, we are finding all those i for which i<=j (Berman codes definition of vecotrs i,j such that i<=j)

In [4]:
#Given a vector j, we are finding all those i for which i<=j (Berman codes definition of vecotrs i,j such that i<=j)
#Param: input vec 
def vectorSubset(vec):
    m = len(vec)
    wt,supp,suppInv = Hweight(vec)
    #if the vector has no weight, then return all zero vector
    if(wt==0):
        #Shape of the array is 1 x M (2-D array of 1 row and M colums), data type is int
        return np.zeros([1, m],int)
    else:
        #Shape of the array is 2**wt x M (2-D array of 2**wt row and M colums), data type is int
        ans = np.empty([2**wt,m],int)
        for i in range(0,2**wt):
            #Convert the number i to binary to know which numbers from the support set are included in the subvector
            pos = decimalToFn(i,2)
            #We are making every pos vector of the same size by padding zeros in the front of the already created pos vector
            padSize = wt - len(pos)
            pos = np.pad(pos,(padSize,0),'constant')

            subVec = np.copy(vec)
            #Here wt = len(pos)
            for j in range(0,len(pos)):
                #Zero in pos vector indicate that we dont have to include those corresponding support elements of vec in the sub vector 
                if(pos[j]==0):
                    subVec[supp[j]] = 0
            ans[i,:] = subVec
        return ans
#print(vectorSubset([0,0, 2,1,3, 0]))

## Starpro: 
Calculating the star product of 2 matrices

In [5]:
#param b1: a 2D vector, b2: a 2D vector
def starPro(b1,b2):
    #checking if the code lengths are equal (shape[1] = no of columns)
    if(b1.shape[1]!=b2.shape[1]):
        return (-1*np.ones([1,b1.shape[1]],int))
    # shape[0] = no of rows which is the dimension of the code
    len1 = b1.shape[0]
    len2 = b2.shape[0]
    if(len1==0 or len2==0):
        return np.zeros([1,b1.shape[1]])
    
    ans = np.empty([len1*len2,b1.shape[1]],int)
    
    for i in range(0,len1):
        for j in range(0,len2):
            a = np.array(b1[i,:])
            b = np.array(b2[j,:])
            ans[i*len2+j,:] = (a*b)
        
    return ans
# a = np.array([[1,2,3,4,5]])
# b = np.array([[1,0,0,0,0]])
# print(starPro(a,b))

## basisBerman: 
Gives the generator matrix of Berman with the input parameters

In [6]:
from math import factorial
import itertools
import numpy as np

def basisBerman(n,m,r):
    #Calculating the dimension of the Code
    dim = 0
    for w in range(r+1,m+1):
        dim+= (factorial(m)/(factorial(w) * factorial(m-w))*(n-1)**w)
    #Creating a basis matrix (Generator matrix)
    basis = np.empty([int(dim),int(n**m)],int)
    basisLen = 0
    for i in range(0,n**m):
        #making i into a m length vector in fn by padding
        vec = decimalToFn(i,n)
        padSize = m - len(vec)
        vec = np.pad(vec,(padSize,0),'constant')
        #We are looking for all those vectors whose support size is > r
        if(Hweight(vec)[0]>=r+1):
            #we are finding all the sub vectors of vec
            sub = vectorSubset(vec)
            #Finding the codeword corresponding to vec
            arr = np.zeros(n**m,int)
            #for each subvector of vec, we are setting its corresponding position in the codeword to 1
            for j in range(0,sub.shape[0]):
                i = FnToDecimal(sub[j,:],n)
                arr[i] = 1
            #we are updating its basis set
            basis[basisLen,:] = arr
            basisLen+=1
    return basis



In [7]:
import numpy as np
n = 3
m = 5

basis1 = basisBerman(n,m,2)
basis2 = basisBerman(n,m,0)

print(basis1.shape[1])
star = starPro(basis1,basis2)
rank = np.linalg.matrix_rank(star)

print("rank is ",rank)

243
rank is  243


## vectorSuperSet

Given a vector j, we are finding all those i for which j<=i (Berman codes definition of vecotrs i,j such that j<=i)

In [8]:
#Given a vector j, we are finding all those i for which j<=i (Berman codes definition of vecotrs i,j such that j<=i)
#Param: input vec, n
import numpy as np

def vectorSuperSet(vec, n):
    m = len(vec)
    wt,supp, suppInv = Hweight(vec)
    
    #if the vector has full weight, then return the vector itself
    if(wt==m):
        #Shape of the array is 1 x M (2-D array of 1 row and M colums), data type is int
        ans = np.zeros([1, m],int)
        ans[0,:] = vec
        return ans
    else:
        #Shape of the array is 2**wt x M (2-D array of 2**wt row and M colums), data type is int
        ans = np.empty([n**(m-wt),m],int)
        for i in range(0,n**(m-wt)):
            #Convert the number i to Fn to know which numbers from Fn are there in the zero positions
            pos = decimalToFn(i,n)
            #We are making every pos vector of the same size by padding zeros in the front of the already created pos vector
            padSize = (m - wt) - len(pos)
            pos = np.pad(pos,(padSize,0),'constant')
            
            subVec = np.copy(vec)
            #Here m-wt = len(pos)
            for j in range(0,len(pos)):
                subVec[suppInv[j]] = pos[j]
            ans[i,:] = subVec
        # print(ans.shape)
        return ans
print(vectorSuperSet([2,1,3], 4))

[[2 1 3]]


In [9]:
from math import factorial
import itertools
import numpy as np

def basisDualBerman(n,m,r):
    #Calculating the dimension of the Code
    dim = 0
    for w in range(0, r+1):
        dim+= (factorial(m)/(factorial(w) * factorial(m-w))*(n-1)**w)
    #Creating a basis matrix (Generator matrix)
    basis = np.empty([int(dim),int(n**m)],int)
    basisLen = 0
    for i in range(0,n**m):
        #making i into a m length vector in fn by padding
        vec = decimalToFn(i,n)
        padSize = m - len(vec)
        vec = np.pad(vec,(padSize,0),'constant')
        #We are looking for all those vectors whose support size is <= r
        if(Hweight(vec)[0]<=r):
            #we are finding all the sub vectors of vec
            sub = vectorSuperSet(vec, n)
            #Finding the codeword corresponding to vec
            arr = np.zeros(n**m,int)
            #for each subvector of vec, we are setting its corresponding position in the codeword to 1
            for j in range(0,sub.shape[0]):
                i = FnToDecimal(sub[j,:],n)
                arr[i] = 1
            #we are updating its basis set
            basis[basisLen,:] = arr
            basisLen+=1
    return dim,basis

In [31]:
import numpy as np
import sympy as sp 

n = 4
m = 5

basis1 = basisDualBerman(n,m,0)[1]
basis2 = basisDualBerman(n,m,2)[1]
basis3 = basisDualBerman(n,m,3)[1]
print(basis1.shape[1])
star = starPro(basis1,basis2)

rank = np.linalg.matrix_rank(star)
starsympy = sp.Matrix(star)
basis3sympy = sp.Matrix(basis3)
star_rref = starsympy.rref()[0]
basis3_rref = basis3sympy.rref()[0]
print("rank is ",rank)
if(rank == basis3_rref.shape[0]):
    dummy = star_rref[:rank, :] - basis3_rref
    if (2 * dummy == dummy):
        print("true")
    else:
        print("false")
else:
    print("Basis ranks are different")

1024
rank is  376
true
